In [1]:
import numpy as np
import pandas as pd
import dicom
import os
import matplotlib.pyplot as plt
import cv2
import math
from sklearn.model_selection import StratifiedKFold
import xgboost as xgb
from scipy.stats import gmean

IMG_SIZE_PX = 50
SLICE_COUNT = 20

In [ ]:
##Visualization
%matplotlib inline
import matplotlib.pyplot as plt
import cv2
import numpy as np

IMG_PX_SIZE = 28

slicePatient = []
count = 0

sampleLabels = []
sample_dir = 'C:/Users/parth/Desktop/PA/Bowl17/Data/sample_images/'


for num,patient in enumerate(patients[:]):
    label = labels_df.get_value(patient,'cancer')
    path = sample_dir + patient
    slices = [dicom.read_file(path + '/' + s) for s in os.listdir(path)]
    slices.sort(key=lambda x:int(x.ImagePositionPatient[2]))
    
    sampleLabels = sampleLabels + [label] * (len(slices))
    #sliceLen = sliceLen+len(slices)
    slicePatient = slicePatient + slices

    
fig = plt.figure()
for num,each_slice in enumerate(slices[:20]):
    y= fig.add_subplot(4,5,num+1)
    new_image = cv2.resize(np.array(each_slice.pixel_array),(IMG_PX_SIZE,IMG_PX_SIZE))
    y.imshow(new_image)

plt.show()  


In [2]:
def chunks(l, n):
    """Yield successive n-sized chunks from l."""
    for i in range(0, len(l), n):
        yield l[i:i + n]
        

def mean(a):
    return sum(a) / len(a)

In [3]:
def process_data(patient,filedir,lab_df,img_px_size=IMG_SIZE_PX, hm_slices=20, visualize=False):
    ##print('Inside process')
    
    label = lab_df.get_value(patient, 'cancer')
    path = filedir + patient
    ##print(path)
    slices = [dicom.read_file(path + '/' + s) for s in os.listdir(path)]
    slices.sort(key = lambda x: int(x.ImagePositionPatient[2]))

    new_slices = []
    slices = [cv2.resize(np.array(each_slice.pixel_array),(img_px_size,img_px_size)) for each_slice in slices]
    
    chunk_sizes = math.ceil(len(slices) / hm_slices)
    for slice_chunk in chunks(slices, chunk_sizes):
        slice_chunk = list(map(mean, zip(*slice_chunk)))
        new_slices.append(slice_chunk)

    if len(new_slices) == hm_slices-1:
        new_slices.append(new_slices[-1])

    if len(new_slices) == hm_slices-2:
        new_slices.append(new_slices[-1])
        new_slices.append(new_slices[-1])

    if len(new_slices) == hm_slices+2:
        new_val = list(map(mean, zip(*[new_slices[hm_slices-1],new_slices[hm_slices],])))
        del new_slices[hm_slices]
        new_slices[hm_slices-1] = new_val
        
    if len(new_slices) == hm_slices+1:
        new_val = list(map(mean, zip(*[new_slices[hm_slices-1],new_slices[hm_slices],])))
        del new_slices[hm_slices]
        new_slices[hm_slices-1] = new_val

    if visualize:
        fig = plt.figure()
        for num,each_slice in enumerate(new_slices):
            y = fig.add_subplot(4,5,num+1)
            y.imshow(each_slice, cmap='gray')
        plt.show()

    if label == 1: label=np.array([0,1])
    elif label == 0: label=np.array([1,0])
        
    return np.array(new_slices),label

In [4]:
##Run Just Once and read from Hard Disk

stage1_dir = 'C:/Users/parth/Desktop/PA/Bowl17/Data/stage1/'
patients = os.listdir(stage1_dir)
stage1_df = pd.read_csv('C:/Users/parth/Desktop/PA/Bowl17/Data/stage1_labels_new.csv', index_col=0)

stage1_data = []
for num,patient in enumerate(patients):
    if num % 100 == 0:
        print(num)
    try:
        img_data,label = process_data(patient,stage1_dir,stage1_df,img_px_size=IMG_SIZE_PX, hm_slices=SLICE_COUNT)
        #print(img_data.shape,label)
        stage1_data.append([img_data,label])
    except KeyError as e:
        print('This is unlabeled data!')

np.save('stage1data-{}-{}-{}.npy'.format(IMG_SIZE_PX,IMG_SIZE_PX,SLICE_COUNT), stage1_data)


"\nstage1_dir = 'C:/Users/parth/Desktop/PA/Bowl17/Data/stage1/'\npatients = os.listdir(stage1_dir)\nstage1_df = pd.read_csv('C:/Users/parth/Desktop/PA/Bowl17/Data/stage1_labels_new.csv', index_col=0)\n\nstage1_data = []\nfor num,patient in enumerate(patients):\n    if num % 100 == 0:\n        print(num)\n    try:\n        img_data,label = process_data(patient,stage1_dir,stage1_df,img_px_size=IMG_SIZE_PX, hm_slices=SLICE_COUNT)\n        #print(img_data.shape,label)\n        stage1_data.append([img_data,label])\n    except KeyError as e:\n        print('This is unlabeled data!')\n\nnp.save('stage1data-{}-{}-{}.npy'.format(IMG_SIZE_PX,IMG_SIZE_PX,SLICE_COUNT), stage1_data)\n"

In [5]:
stage1_data = np.load('stage1data-50-50-20.npy')

train_data = stage1_data[:]
print(train_data.shape)

train_features = []
train_labels = []


for i in range(0,train_data.shape[0]):
    train_features = train_features + [(train_data[i,0])[0]]
    train_labels = train_labels + [(train_data[i,1])[1]]
'''
for i in range(0,len(train_features)):
    for j in range(0,len(train_features[i])):
        for k in range(0,len((train_features[i])[j])):
                       if ((train_features[i])[j])[k] == -2000:
                           ((train_features[i])[j])[k] = 0
'''    
train_features = np.array(train_features)
train_labels = np.array(train_labels)

nsamples, nx, ny = train_features.shape
d2_train_features = train_features.reshape((nsamples,nx*ny))
'''
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler().fit(d2_train_features)
d2_train_features = scaler.transform(d2_train_features)
'''
print(train_features.shape)
print(d2_train_features.shape)
print(train_labels.shape)

del train_data
del stage1_data
del train_features

(1595, 2)
(1595, 50, 50)
(1595, 2500)
(1595,)


In [6]:
##Run Just Once and read from Hard Disk

stage2_dir = 'C:/Users/parth/Desktop/PA/Bowl17/Data/stage2/'
patients = os.listdir(stage2_dir)
stage2_df = pd.read_csv('C:/Users/parth/Desktop/PA/Bowl17/Data/stage2_sample_submission.csv', index_col=0)

testPatients_df = pd.read_csv('C:/Users/parth/Desktop/PA/Bowl17/Data/stage2_sample_submission.csv')
testPatientsList = testPatients_df['id'].tolist()

stage2_data = []
for num,patient in enumerate(testPatientsList):
    if num % 100 == 0:
        print(num)
    try:
        img_data,label = process_data(patient,stage2_dir,stage2_df,img_px_size=IMG_SIZE_PX, hm_slices=SLICE_COUNT)
        ##print(str(num) + '    Back to try')
        stage2_data.append([img_data,label])
    except KeyError as e:
        print('This is unlabeled data!')

np.save('stage2data-{}-{}-{}.npy'.format(IMG_SIZE_PX,IMG_SIZE_PX,SLICE_COUNT), stage2_data)


"\nstage2_dir = 'C:/Users/parth/Desktop/PA/Bowl17/Data/stage2/'\npatients = os.listdir(stage2_dir)\nstage2_df = pd.read_csv('C:/Users/parth/Desktop/PA/Bowl17/Data/stage2_sample_submission.csv', index_col=0)\n\ntestPatients_df = pd.read_csv('C:/Users/parth/Desktop/PA/Bowl17/Data/stage2_sample_submission.csv')\ntestPatientsList = testPatients_df['id'].tolist()\n\nstage2_data = []\nfor num,patient in enumerate(testPatientsList):\n    if num % 100 == 0:\n        print(num)\n    try:\n        img_data,label = process_data(patient,stage2_dir,stage2_df,img_px_size=IMG_SIZE_PX, hm_slices=SLICE_COUNT)\n        ##print(str(num) + '    Back to try')\n        stage2_data.append([img_data,label])\n    except KeyError as e:\n        print('This is unlabeled data!')\n\nnp.save('stage2data-{}-{}-{}.npy'.format(IMG_SIZE_PX,IMG_SIZE_PX,SLICE_COUNT), stage2_data)\n"

In [7]:
stage2_data = np.load('stage2data-50-50-20.npy')

print('Test Data')
test_data = stage2_data[:]
print(test_data.shape)

test_features = []

for i in range(0,test_data.shape[0]):
    test_features = test_features + [(test_data[i,0])[0]]
'''
for i in range(0,len(test_features)):
    for j in range(0,len(test_features[i])):
        for k in range(0,len((test_features[i])[j])):
                       if ((test_features[i])[j])[k] == -2000:
                           ((test_features[i])[j])[k] = 0    
'''    
test_features = np.array(test_features)
nsamples, nx, ny = test_features.shape
d2_test_features = test_features.reshape((nsamples,nx*ny))
'''
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler().fit(d2_test_features)
d2_test_features = scaler.transform(d2_test_features)
'''
print('Test Data')
print(test_features.shape)
print(d2_test_features.shape)

del stage2_data
del test_data
del test_features

Test Data
(506, 2)
Test Data
(506, 50, 50)
(506, 2500)


In [8]:
from sklearn.linear_model import LogisticRegression

from sklearn.grid_search import GridSearchCV
#Set the tuning parameter
params = {'C':[1000,1200,1400], 'tol': [0.00001]}

##Create the Logistic Regression model
logreg = LogisticRegression(solver='newton-cg', multi_class='multinomial')
##Use the Grid Search with the params
clf = GridSearchCV(logreg,params, scoring='log_loss', refit='True', n_jobs=-1, cv=5)
##Fit the models
clf_fit = clf.fit(d2_train_features,train_labels)

test_pred_labels= clf.predict(d2_test_features)

test_pred_labProb = clf.predict_proba(d2_test_features)
pred = []
for i in range(0,test_pred_labProb.shape[0]):
    pred = pred + [test_pred_labProb[i,1]]
#print(pred)
'''
test_pred_logProb = clf.predict_log_proba(d2_test_features)
pred = []
for i in range(0,test_pred_logProb.shape[0]):
    pred = pred + [test_pred_logProb[i,1]]
'''
logisticPatients_df = pd.read_csv('C:/Users/parth/Desktop/PA/Bowl17/Data/stage2_sample_submission.csv')
logisticPatients_df['cancer'] = pd.Series(pred)
logisticPatients_df.to_csv("C:/Users/parth/Desktop/PA/Bowl17/Data/stage2_pred_log_final.csv",index=False)

C:\Users\parth\Anaconda3\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
C:\Users\parth\Anaconda3\lib\site-packages\sklearn\grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [10]:
def train_my_xgb():
    df = pd.read_csv('C:/Users/parth/Desktop/PA/Bowl17/Data/stage1_labels_new.csv')

    x = d2_train_features##np.load('stage1data-50-50-20.npy')
    y = train_labels

    clfs = []
    for seed in range(15):

        skf = StratifiedKFold(n_splits=10, random_state=14+seed, shuffle=True)
    
        
        '''
        for train_index, test_index in skf.split(x, y):
            trn_x, val_x = x[train_index,:], x[test_index,:]
            trn_y, val_y = y[train_index], y[test_index]
        '''
        clf = xgb.XGBRegressor(max_depth=11,
                               n_estimators=1500,
                               min_child_weight=9,
                               learning_rate=0.03,
                               nthread=8,
                               subsample=0.80,
                               colsample_bytree=0.80,
                               seed=88+seed)

        clf.fit(d2_train_features, train_labels)
        clfs.append(clf)

    return clfs

In [11]:
clfs = train_my_xgb()

df = pd.read_csv('C:/Users/parth/Desktop/PA/Bowl17/Data/stage2_sample_submission.csv')

x = d2_test_features
preds = []
for clf in clfs:
    preds.append(np.clip(clf.predict(x),0.001,1))

pred = gmean(np.array(preds), axis=0)
df['cancer'] = pred
df.to_csv('C:/Users/parth/Desktop/PA/Bowl17/Data/stage2_pred_xgb_final.csv', index=False)
print(df.head())

                                 id    cancer
0  004828796b994741c4466f59a8c7e9a4  0.224903
1  007c1246c5fe6f200378f6b91323dc2a  0.390215
2  00f6c1bd02eb49e3d8dbfc7d957a709e  0.391744
3  0171e54e4c0f68e8fc8c24523f71a86f  0.273023
4  019cb268efb93de5446984242bac0380  0.354794
